In [1]:
import sqlite3
conn = sqlite3.connect('data/WDIProd.sqlite')


In [2]:
def getMaxMinForData(row):
    return r'SELECT max("{}") as maxvalue, min("{}") as minValue, avg("{}") as avgvalue FROM "data"'.format(row, row, row)

In [3]:
getMaxMinForData("Test")

'SELECT max("Test") as maxvalue, min("Test") as minValue, avg("Test") as avgvalue FROM "data"'

In [4]:
class DataSelector:
    def __init__(self, yearDelta, metrics, note = ""):
        self.yearDelta = yearDelta
        self.metrics = metrics
        self.note = note
    
class QueryConstructor:
    def __init__(self, dataSelector):
        self.dataSelector = dataSelector
        
    def formartTableName(self, table, row):
        return r'"{}"."{}"'.format(table,row)
        
    def formartName(self, table):
        return r'"{}"'.format(table)
        
    def constructForYear(self, year):
        output = ""
        
        #SELECT PART FIRST
        output += r'SELECT '
        
        for selectedData in self.dataSelector.metrics:
            output += self.formartTableName("time1",selectedData)
            output += " as "
            output += self.formartName("p."+selectedData)
            output += " , "
        
        output += " "
        
        #Country Name as N row
        output += self.formartTableName("origins","CountryCode")
        output += ", "
        
        #T+0 GDP as N+1 row
        output += self.formartTableName("time2","NY.GDP.PCAP.PP.KD")
        output += " as gdpf ,"
        
        #T+yY GDP as N+2 row
        output += self.formartTableName("time1","NY.GDP.PCAP.PP.KD")
        output += " as gdpp "
            
        year2 = year
        year1 = year - self.dataSelector.yearDelta
        
        data = r' from (SELECT DISTINCT "origin"."CountryCode" from "data" as "origin") as "origins" LEFT JOIN "data" as "time1" ON "time1"."CountryCode" == "origins"."CountryCode" AND "time1"."Year" == {} LEFT JOIN "data" as "time2" ON "time2"."CountryCode" == "origins"."CountryCode" AND "time2"."Year" == {} WHERE gdpf is not NULL AND gdpp is not NULL'.format(year1,year2)
        
        output += data
        
        return output

In [5]:
# SELECT "origin1"."CountryCode", "time1"."EN.ATM.NOXE.KT.CE" as "time1.EN.ATM.NOXE.KT.CE", "time2"."EN.ATM.NOXE.KT.CE" as "time2.EN.ATM.NOXE.KT.CE" from (SELECT DISTINCT "origin"."CountryCode" from "data" as "origin") as "origin1" LEFT JOIN "data" as "time1" ON "time1"."CountryCode" == "origin1"."CountryCode" AND "time1"."Year" == 1990 LEFT JOIN "data" as "time2" ON "time2"."CountryCode" == "origin1"."CountryCode" AND "time2"."Year" == 1991

In [6]:
ds = DataSelector(5,["EN.MAM.THRD.NO","BX.TRF.PWKR.DT.GD.ZS"])

In [7]:
qc = QueryConstructor(ds)

In [8]:
qc.constructForYear(2018)

'SELECT "time1"."EN.MAM.THRD.NO" as "p.EN.MAM.THRD.NO" , "time1"."BX.TRF.PWKR.DT.GD.ZS" as "p.BX.TRF.PWKR.DT.GD.ZS" ,  "origins"."CountryCode", "time2"."NY.GDP.PCAP.PP.KD" as gdpf ,"time1"."NY.GDP.PCAP.PP.KD" as gdpp  from (SELECT DISTINCT "origin"."CountryCode" from "data" as "origin") as "origins" LEFT JOIN "data" as "time1" ON "time1"."CountryCode" == "origins"."CountryCode" AND "time1"."Year" == 2013 LEFT JOIN "data" as "time2" ON "time2"."CountryCode" == "origins"."CountryCode" AND "time2"."Year" == 2018 WHERE gdpf is not NULL AND gdpp is not NULL'

In [9]:
list(conn.execute(qc.constructForYear(2018)))

[(None, 1.68845930330232, 'AFG', 2190.24032132044, 2264.3175752325),
 (None, 0.0267993914860461, 'AGO', 6933.50930820608, 8140.53120789329),
 (None, 10.0330801692658, 'ALB', 13601.3033805754, 11597.9632735891),
 (None, 2.07731781241171, 'ARB', 14620.2610718972, 14136.9464230217),
 (None, None, 'ARE', 66968.2698548892, 59921.7348033071),
 (None, 0.0969166746867399, 'ARG', 22745.9037844103, 24424.3504112585),
 (None, 19.7113750362956, 'ARM', 12714.9581875586, 10691.3099550604),
 (None, 1.78825882234923, 'ATG', 21115.7982687308, 17641.5425333279),
 (None, 0.15159377815888, 'AUS', 49575.981131221, 47192.0573890772),
 (None, 0.758368508471961, 'AUT', 55687.1892771302, 53289.3783085002),
 (None, 2.33692601835619, 'AZE', 14209.6494059309, 14652.7042457179),
 (None, 1.98396848206385, 'BDI', 761.524193384446, 877.274160261081),
 (None, 2.24197236441209, 'BEL', 51245.9962234656, 48243.0201797298),
 (None, 1.98756670280948, 'BEN', 3160.7770925266, 2876.90563418335),
 (None, 2.29442584641875, 'BFA

In [10]:
def Normalize(meanv, minmaxdiffv):
    """
    :param a: Array
    :return: Array
    """
    mean = meanv
    maxmindiff = minmaxdiffv

    def apply(v):
        if v is None:
            return 0
        return (v - mean) / maxmindiff

    def apply_inversed(v):
        return v * maxmindiff + mean

    return apply, apply_inversed


class DataFromSelector:
    def __init__(self, dataSelector, yearmin, years, conn):
        self.dataSelector = dataSelector
        self.yearmin = yearmin
        self.years = years # The window of years
        
        self.qc = QueryConstructor(self.dataSelector)
        self.conn = conn
        
        
        normalF = []
        for mat in self.dataSelector.metrics:
            nf = self.getNormalizeFunction(mat)[0]
            normalF.append(nf)
        
        self.normalF = normalF
        
    def getNormalizeFunction(self,metric):
        sql = getMaxMinForData(metric)
        
        maxmin = self.conn.execute(sql).fetchone()
        
        maxv = maxmin[0]
        minv = maxmin[1]
        
        avgv = maxmin[2]
        
        return Normalize(avgv,maxv-minv)
        
    def getProsperityIndex(self, gdp1, gdp2):
        if gdp2-gdp1 > 0:
            return pow((gdp2-gdp1)/gdp1,(1/self.years))
        else:
            return - pow((gdp1-gdp2)/gdp1,(1/self.years))
    
    def getXfromRow(self, row):
        rawData = row[:-3]
        normalF = self.normalF
        
        data = []
        for item in range(len(rawData)):
                data.append(normalF[item](rawData[item]))
        return data
    
    def getYfromRow(self, row):
        return self.getProsperityIndex(row[-2],row[-1])
        
        #return (row[-2],row[-1])
    
    def constructOneYear(self, year, X, Y):
        sql = self.qc.constructForYear(year)
        res = self.conn.execute(sql)
        for data in res:
            X.append(self.getXfromRow(data))
            Y.append(self.getYfromRow(data))
    
    def constructAll(self):
        X = []
        Y = []
        
        for ye in range(self.yearmin, self.yearmin+self.years):
            self.constructOneYear(ye, X,Y)
        
        return (X,Y)

In [11]:
dfs = DataFromSelector(ds,2000,15,conn)

In [12]:
len(dfs.constructAll()[1])

3435


Size of the economy (GDP per capita) (http://wdi.worldbank.org/table/WV.1) NY.GDP.PCAP.PP.KD
Poverty rates at international poverty lines (http://wdi.worldbank.org/table/1.2) SI.POV.LMIC.GP SI.POV.GINI
Distribution of income or consumption (http://wdi.worldbank.org/table/1.3) SI.DST.10TH.10 SI.DST.05TH.20 SI.DST.FRST.10 SI.DST.FRST.20 SI.DST.50MD
Labor force structure (http://wdi.worldbank.org/table/2.2)  SL.TLF.0714.ZS SL.EMP.1524.SP.ZS  SL.TLF.ADVN.ZS SL.TLF.BASC.ZS SL.TLF.INTM.ZS
Employment by sector (http://wdi.worldbank.org/table/2.3) SL.IND.EMPL.ZS SL.ISV.IFRM.ZS  SL.EMP.WORK.ZS SL.AGR.EMPL.ZS 
Unemployment (http://wdi.worldbank.org/table/2.5) SL.UEM.1524.ZS SL.UEM.TOTL.ZS
Health systems (http://wdi.worldbank.org/table/2.12) SP.DYN.LE00.IN SH.XPD.OOPC.CH.ZS SH.XPD.CHEX.PP.CD
Energy dependency, efficiency and carbon dioxide emissions (http://wdi.worldbank.org/table/3.8)
Structure of merchandise exports (http://wdi.worldbank.org/table/4.4)
Structure of merchandise imports (http://wdi.worldbank.org/table/4.5)
Structure of service exports (http://wdi.worldbank.org/table/4.6)
Structure of service imports (http://wdi.worldbank.org/table/4.7)
Business environment: Doing Business indicators (http://wdi.worldbank.org/table/5.3)
Financial access, stability and efficiency (http://wdi.worldbank.org/table/5.5)
Tax policies (http://wdi.worldbank.org/table/5.6)

In [13]:
ds = DataSelector(5,["NY.GDP.PCAP.PP.KD","SI.POV.LMIC.GP","SI.DST.10TH.10","SI.DST.50MD"])

In [14]:
dfs = DataFromSelector(ds,1970,45,conn)

In [15]:
len(dfs.constructAll()[1])

4480

In [16]:
class CombinedDataSelector:
    def __init__(self,daArray):
        self.daArray = daArray
        
    def Construct():
        for itemi in range(self.daArray):
            cloned = list(self.daArray)
            cloned.remove(itemi)
            selected = []
            for v in cloned:
                selected.extend(v.metrics)
            yield (selected,itemi)
        

In [17]:
import numpy as np
from models import LinRegWithPoly, RidgeRegCVWithPoly, LassoCVWithPoly
from sklearn.model_selection import train_test_split

In [20]:
def SplitTestEval(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(np.array(dfs.constructAll()[0]),
     np.array(dfs.constructAll()[1]), test_size=0.1, random_state=42)

In [32]:
lr_model = LinRegWithPoly(power=5)
lr_model.fit(X_train, y_train)
lr_model.evaluate(X_test, y_test)

(0,
 {'kfold_mse': 1.166292117255559,
  'kfold_std': 0.6137853705586909,
  'val_mse': 8.205107165994354,
  'val_rmse': 2.8644558237114346,
  'val_r2': -16.389207484996646})

In [33]:
lr_model.coef_

array([-3.36585667e-08, -4.90442834e-01, -2.51749366e+00,  6.54424100e+00,
        1.16862538e+00,  1.47664046e+01,  8.27482026e+00, -4.44248595e+01,
        1.52441955e+02, -3.36630519e+00, -2.74534197e+01,  4.87637727e+01,
        8.26503379e+00,  2.78110671e+01, -5.08323394e+01, -2.14829494e+01,
        1.39531719e+03, -4.00868989e+03,  2.31669159e+03,  6.49793906e+02,
       -2.06381663e+03,  1.38818934e+03, -1.07187763e+03,  2.87666800e+03,
       -1.80935887e+03,  1.64661651e+02, -4.67628925e+02,  1.27276857e+02,
       -2.33796856e+02,  7.21653977e+02, -4.67325982e+02, -1.48824873e+02,
        1.49110069e+02, -8.42826652e+01,  5.39148236e+01, -3.55356688e+01,
        1.03836906e+04, -3.39162573e+04,  1.02249647e+04,  1.13590026e+04,
       -3.48723857e+04,  1.33106412e+04, -1.89054352e+04,  3.72009317e+04,
       -1.78304490e+04,  2.39095788e+03, -9.62580778e+03, -7.82946477e+02,
       -1.27625222e+04,  3.15598745e+04, -1.52443933e+04,  1.42404395e+01,
       -3.10616807e+03,  

In [34]:
ridge = RidgeRegCVWithPoly(power=5, alphas=[1e-3, 1e-2, 1e-1, 1, 10])

In [35]:
ridge.fit(X_train, y_train)
ridge.evaluate(X_test, y_test)

(0,
 {'kfold_mse': 0.49128709144489635,
  'kfold_std': 0.051186716282342615,
  'val_mse': 0.4754208794378205,
  'val_rmse': 0.6895077080336525,
  'val_r2': -0.007566646966756352})

In [36]:
lasso = LassoCVWithPoly(power=5, alphas=[1e-3, 1e-2, 1e-1, 1, 10])

In [37]:
lasso.fit(X_train, y_train)
lasso.evaluate(X_test, y_test)

(0,
 {'kfold_mse': 0.4920128907877438,
  'kfold_std': 0.041959898695600635,
  'val_mse': 0.46812271363514996,
  'val_rmse': 0.684194938329092,
  'val_r2': 0.007900465995340555})

!pip install sklearn

!pip install sklearn

def TrainingPipeline(daArray, conn, model):
    cds = CombinedDataSelector(daArray)
    for s in cds.Construct():
        (selected,itemi) = s
        ds = DataSelector(5,selected)
        dfs = DataFromSelector(ds,1980,25,conn)
        datas = dfs.constructAll()
        (X, y) = datas
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        modelc = model.getNewConsolitedModel()
        modelc.fit(X_train,y_train)
        Score = modelc.evaluate(X_test,y_test)[0]
        print(itemi.note, Score)
    